# Dataset, Dataloader, BatchNorm, Dropout, Оптимизация

Сегодня у нас довольно объемная программа, она призвана закончить большую часть про базовую функциональность Pytorch, дальше мы двинемся к сверточным сетям и будем решать более прикладные задачи

# Домашнее задание

Будем практиковаться на датасете недвижимости (sklearn.datasets.fetch_california_housing)

Ваша задача:
1. Создать Dataset для загрузки данных
2. Обернуть его в Dataloader
3. Написать архитектуру сети, которая предсказывает стоимость недвижимости. Сеть должна включать BatchNorm слои и Dropout (или НЕ включать, но нужно обосновать)
4. Сравните сходимость Adam, RMSProp и SGD, сделайте вывод по качеству работы модели

train-test разделение нужно сделать с помощью sklearn random_state=13, test_size = 0.25

## Dataset

In [1]:
import math
import torch
import numpy as np

# from torchvision import transforms, datasets
from sklearn.preprocessing import MinMaxScaler

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split


In [3]:
from torchmetrics import MeanAbsolutePercentageError

In [4]:
import torch.nn.functional as F
import torch.nn as nn

In [5]:
class Perceptron(nn.Module):
    def __init__(self, input_dim, output_dim, activation="relu"):
        super().__init__()
        self.fc = nn.Linear(input_dim, output_dim)
        self.activation = activation
        
    def forward(self, x):
        x = self.fc(x)
        if self.activation == "tanh":
            return F.tanh(x)
        if self.activation == "relu":
            return F.relu(x)
        raise RuntimeError
        

class FeedForward(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.fc1 = Perceptron(input_dim, hidden_dim)
        self.bn = nn.BatchNorm1d(hidden_dim)
        self.dp = nn.Dropout(0.25)
        self.fc2 = Perceptron(hidden_dim, 1, "relu")
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.dp(x)
        x = self.bn(x)
        x = self.fc2(x)
        return x

In [6]:
class CalifHousingDataset(torch.utils.data.Dataset):
   
    def __init__(self, init_features, init_target, transform=None):
        self._features_data = torch.tensor(init_features, dtype=torch.float32)
        self._target_data = torch.tensor(init_target, dtype=torch.float32)
        self.transform = transform

    def __len__(self):
        return len(self._features_data)

    def __getitem__(self, idx):
        item_features = self._features_data[idx]
        item_target = self._target_data[idx]
      
        return item_features, item_target
    
#trans_actions = transforms.Compose([transforms.ToTensor()])


In [7]:
housing_data = fetch_california_housing()

In [8]:
features, target = housing_data['data'], housing_data['target']

In [9]:
scaler_f = MinMaxScaler()

In [10]:
scaled_features = scaler_f.fit_transform(features)

In [11]:
x_train, x_test, y_train, y_test = train_test_split(scaled_features, target, test_size=0.25, random_state=13)

In [12]:
x_train.shape

(15480, 8)

In [13]:
dataset_train = CalifHousingDataset(x_train, y_train)

In [14]:
dataset_train[0]

(tensor([0.2081, 0.6863, 0.0262, 0.0226, 0.0379, 0.0011, 0.1775, 0.6096]),
 tensor(2.6800))

In [15]:
dataset_test = CalifHousingDataset(x_test, y_test)

In [16]:
train_loader = torch.utils.data.DataLoader(dataset_train,
                          batch_size=128, drop_last=True,
                          shuffle=True)

In [17]:
test_loader = torch.utils.data.DataLoader(dataset_test,
                          batch_size=1,
                          shuffle=False)

In [18]:
net = FeedForward(8, 20)

optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
criterion = nn.MSELoss()

In [19]:
mean_abs_percentage_error = MeanAbsolutePercentageError()


In [21]:
num_epochs = 10

for epoch in range(num_epochs):  
    running_loss, running_items, running_mse, running_mape = 0.0, 0.0, 0.0, 0.0
    for i, data in enumerate(train_loader):
        inputs, targets = data[0], data[1]
        inputs = inputs.to(torch.float32)

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        running_items += len(targets)
        running_mse += (running_loss/running_items)
        running_mape += mean_abs_percentage_error(outputs.view(128), targets) / running_items
        
        # выводим статистику о процессе обучения
        if i % 50 == 0:    # печатаем каждые 50 mini-batches
            net.eval()
            
            print(f'Epoch [{epoch + 1}/{num_epochs}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {running_mse:.3f}. ' \
                  f'MAPE: {running_mape:.3f}. ' , end='. ')
              
            running_loss, running_items, running_mse, running_mape = 0.0, 0.0, 0.0, 0.0

            test_running_right, test_running_total, test_running_mape = 0.0, 0.0, 0.0
            for i, data in enumerate(test_loader):
                            
                test_outputs = net(data[0].to(torch.float32))
                test_running_total += len(data[1])
                test_running_right += loss.item()
                test_running_mape += mean_abs_percentage_error(test_outputs.view(1), data[1])
            
            print(f'Test acc: {test_running_right / test_running_total:.3f}. ' \
                  f'Test MAPE: {test_running_mape  / test_running_total:.3f}. ' , end='\n ')
            net.train()
        
print('Training is finished!')

Epoch [1/10]. Step [1/120]. Loss: 0.008. MAPE: 0.004. . Test acc: 1.079Test MAPE: 0.587. 
 Epoch [1/10]. Step [51/120]. Loss: 0.512. MAPE: 0.021. . Test acc: 1.079Test MAPE: 0.619. 
 Epoch [1/10]. Step [101/120]. Loss: 0.515. MAPE: 0.022. . Test acc: 1.543Test MAPE: 0.603. 
 Epoch [2/10]. Step [1/120]. Loss: 0.012. MAPE: 0.005. . Test acc: 1.529Test MAPE: 0.612. 
 Epoch [2/10]. Step [51/120]. Loss: 0.509. MAPE: 0.023. . Test acc: 1.413Test MAPE: 0.642. 
 Epoch [2/10]. Step [101/120]. Loss: 0.515. MAPE: 0.022. . Test acc: 1.140Test MAPE: 0.627. 
 Epoch [3/10]. Step [1/120]. Loss: 0.011. MAPE: 0.005. . Test acc: 1.354Test MAPE: 0.591. 
 Epoch [3/10]. Step [51/120]. Loss: 0.521. MAPE: 0.022. . Test acc: 1.293Test MAPE: 0.605. 
 Epoch [3/10]. Step [101/120]. Loss: 0.529. MAPE: 0.022. . Test acc: 1.140Test MAPE: 0.629. 
 Epoch [4/10]. Step [1/120]. Loss: 0.010. MAPE: 0.005. . Test acc: 1.299Test MAPE: 0.660. 
 Epoch [4/10]. Step [51/120]. Loss: 0.541. MAPE: 0.023. . Test acc: 1.168Test MAPE